In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def formating_unc(value, unc, title):
    structure = '{0:.7f} ± {1:.7f}'
    print(title)
    for i,j in zip(value,unc):
        print(structure.format(i,j))
        
def func_pressure(force,force_unc,area,area_unc,pressure_atm):
    pressure = force/area + pressure_atm
    pressure_unc_pct = np.sqrt((force_unc/force)**2+(area_unc/area)**2)
    pressure_unc = pressure * pressure_unc_pct
    return pressure, pressure_unc

def func_area()

### Given values

In [5]:
# all lengths in units of m
tube_length = [0.535, 0.0001]
tube_diameter = [0.007, 0.0001]
cylinder_length = [0.16, 0.0001]
cylinder_diameter = [0.039, 0.0001]
piston_length = [.10, 0.0001]
piston_diameter = [0.0159, 0.0001]
pressure_atm = 101325   # units in (kg)/(m*s^2)
mass = [0.0499, 0.0001]
mass_piston_platform = [0.0159, 0.00006]

In [17]:
def func_force(mass,mass_unc,g=9.80):
    force = mass*g
    force_unc = (mass_unc/mass)*force
    return [force, force_unc]
force = func_force(mass[0], mass[1])

In [8]:
height1 = np.array([0.080, 0.040, 0.085, 0.098])
height2 = np.array([0.080, 0.041, 0.071, 0.078])
height3 = np.array([0.080, 0.035, 0.055, 0.065])
height_unc = np.empty(4); height_unc.fill(0.0001)

In [ ]:
area1 = 

In [10]:
points = ['A', 'B', 'C', 'D']
df=pd.DataFrame(data={'Points': points,
                      'Piston Height 1': height1,
                      'Piston Height 2': height2,
                      'Piston Height 3': height3})
df

,Points,Piston Height 1,Piston Height 2,Piston Height 3
0,A,0.080,0.080,0.080
1,B,0.040,0.041,0.035
2,C,0.085,0.071,0.055
3,D,0.098,0.078,0.065
